In [16]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import astropy.units as u
import astropy.constants as const

In [17]:
def surface_grav(mass, radius):

    return (const.G * mass) / radius**2


def a_contraction(r_initial, r_final, time):

    return 2 * (r_final - r_initial) / time.to(u.s) ** 2

In [18]:
g_stara_initial = surface_grav(1.0 * const.M_sun, 7.0 * const.R_sun)
g_stara_final = surface_grav(1.0 * const.M_sun, 0.87 * const.R_sun)
a_stara = a_contraction(7.0 * const.R_sun, 0.87 * const.R_sun, 30e6 * u.year)
print(g_stara_initial, g_stara_final, a_stara)

5.59592064606884 m / s2 362.2672898102433 m / s2 -9.516166452828988e-21 m / s2


In [19]:
g_starb_final = surface_grav(1.0 * const.M_sun, 1 * const.R_sun)
g_starb_initial = surface_grav(1.0 * const.M_sun, 0.87 * const.R_sun)
a_starb = a_contraction(0.87 * const.R_sun, 1 * const.R_sun, 4.57e9 * u.year)
print(g_starb_initial, g_starb_final, a_starb)

362.2672898102433 m / s2 274.20011165737316 m / s2 8.69671057394294e-27 m / s2


## Problem 2

In [20]:
def stellar_energy_by_phase(initial_mass_msun):

    mass_star = initial_mass_msun * const.M_sun\
        # assumed solar
    mass_of_hydrogen = 0.7 * mass_star
    mass_of_helium = 0.3 * mass_star
    
    
    # efficiencies
    eta_H = 0.00712  # H -> He
    eta_He = 0.00083  # He -> C/O

    # masses burned per phase
    f_nuc = 0.1
    m_MS = f_nuc * mass_of_hydrogen # ms
    
    # updates masses
    mass_of_hydrogen_after_MS = mass_of_hydrogen - m_MS
    mass_of_helium_after_MS = mass_of_helium + m_MS * (1 -eta_H) # the helium mass increases
    
    print("after MS")
    print("mass of hydrogen {:.4f} Msun".format(mass_of_hydrogen_after_MS / const.M_sun))
    print("mass of helium {:.4f} Msun".format(mass_of_helium_after_MS / const.M_sun))
    E_MS = eta_H * m_MS * const.c**2
    
    
    # the red giant branch burns a total of 0.5 solar masses of hydrogen,
    # so we need to subtract the mass of hydrogen that was burned in the MS
    # phase from the mass of hydrogen, that's the mass that will be burned in the RGB phase
    mass_for_rgb = (0.5 *const.M_sun) - m_MS
    E_RGBshell = eta_H * mass_for_rgb * const.c**2
    print("mass for RGB shell burning {:.4f} Msun".format(mass_for_rgb / const.M_sun))
    # RGB mass increases the mass of helium and drains the mass of hydrogen
    mass_of_hydrogen_after_RGB = mass_of_hydrogen_after_MS - mass_for_rgb
    mass_of_helium_after_RGB = mass_of_helium_after_MS + mass_for_rgb * (1 - eta_H)

   
    print("after RGB")
    print("mass of hydrogen {:.4f} Msun".format(mass_of_hydrogen_after_RGB / const.M_sun))
    print("mass of helium {:.4f} Msun".format(mass_of_helium_after_RGB / const.M_sun))
    
    # ok, now core helium is burnin gfrom the build up of helium
    # the helium burnin goges on until 0.5 Msun of helium is burned
    mass_for_helium_burning = 0.5 * const.M_sun
    E_He = eta_He * mass_for_helium_burning * const.c**2
    
    # now burn hydrogen so that Luminosity is twice the core helium burning
    # luminosity, so 2 * E_He= 2 * eta_He * mass_for_helium_burning * c^2
    # we need to find how much hydrogen mass is needed to burn this energy 
    H_mass_for_shell_burning_during_CHeB = (2 * E_He) / (eta_H * const.c**2)
    E_shell_burning = eta_H * H_mass_for_shell_burning_during_CHeB * const.c**2
    
    E_total_during_CHeB = E_He + E_shell_burning
    
    # this is the mass of hydrogen that will be burned in the shell during
    # the core helium burning phase, so we need to subtract this from the
    # mass of hydrogen after the red giant branch
    mass_of_hydrogen_after_CHeB = mass_of_hydrogen_after_RGB - H_mass_for_shell_burning_during_CHeB
    
    # now we need to find the mass of helium after the core helium burning phase
    mass_of_helium_after_CHeB = mass_of_helium_after_RGB + H_mass_for_shell_burning_during_CHeB * (1 - eta_H)  - mass_for_helium_burning
    
    # mass of the metals from the helium burning core
    mass_of_metals = mass_for_helium_burning * (1 - eta_He)
    
    print("after CHeB")
    print("mass of hydrogen {:.6f} Msun".format(mass_of_hydrogen_after_CHeB / const.M_sun))
    print("mass of helium {:.6f} Msun".format(mass_of_helium_after_CHeB / const.M_sun))
    print("mass of metals {:.6f} Msun".format(mass_of_metals / const.M_sun))
    

    E_total = E_MS + E_RGBshell + E_total_during_CHeB
    
    m_total= mass_of_hydrogen_after_CHeB + mass_of_helium_after_CHeB + mass_of_metals
    print("mass of star {:.6f} Msun".format(m_total / const.M_sun))
    
    return {
        "Main Sequence": E_MS.to(u.erg),
        "Red Giant Branch":   E_RGBshell.to(u.erg),
        "Core Helium-Burning":  E_total_during_CHeB.to(u.erg),
        "Total": E_total.to(u.erg),
    }


In [21]:
energies = stellar_energy_by_phase(1.0)
for phase, energy in energies.items():
    print(f"{phase}: {energy:.2e}, percent: {energy / energies['Total']:.2%}")

after MS
mass of hydrogen 0.6300 Msun
mass of helium 0.3695 Msun
mass for RGB shell burning 0.4300 Msun
after RGB
mass of hydrogen 0.2000 Msun
mass of helium 0.7964 Msun
after CHeB
mass of hydrogen 0.083427 Msun
mass of helium 0.412183 Msun
mass of metals 0.499585 Msun
mass of star 0.995195 Msun
Main Sequence: 8.91e+50 erg, percent: 10.37%
Red Giant Branch: 5.47e+51 erg, percent: 63.72%
Core Helium-Burning: 2.22e+51 erg, percent: 25.91%
Total: 8.59e+51 erg, percent: 100.00%


In [22]:
energies = stellar_energy_by_phase(2.0)
for phase, energy in energies.items():
    print(f"{phase}: {energy:.2e}, percent: {energy / energies['Total']:.2%}")

after MS
mass of hydrogen 1.2600 Msun
mass of helium 0.7390 Msun
mass for RGB shell burning 0.3600 Msun
after RGB
mass of hydrogen 0.9000 Msun
mass of helium 1.0964 Msun
after CHeB
mass of hydrogen 0.783427 Msun
mass of helium 0.712183 Msun
mass of metals 0.499585 Msun
mass of star 1.995195 Msun
Main Sequence: 1.78e+51 erg, percent: 20.75%
Red Giant Branch: 4.58e+51 erg, percent: 53.34%
Core Helium-Burning: 2.22e+51 erg, percent: 25.91%
Total: 8.59e+51 erg, percent: 100.00%


In [23]:
energies = stellar_energy_by_phase(4.0)
for phase, energy in energies.items():
    print(f"{phase}: {energy:.2e}, percent: {energy / energies['Total']:.2%}")

after MS
mass of hydrogen 2.5200 Msun
mass of helium 1.4780 Msun
mass for RGB shell burning 0.2200 Msun
after RGB
mass of hydrogen 2.3000 Msun
mass of helium 1.6964 Msun
after CHeB
mass of hydrogen 2.183427 Msun
mass of helium 1.312183 Msun
mass of metals 0.499585 Msun
mass of star 3.995195 Msun
Main Sequence: 3.56e+51 erg, percent: 41.49%
Red Giant Branch: 2.80e+51 erg, percent: 32.60%
Core Helium-Burning: 2.22e+51 erg, percent: 25.91%
Total: 8.59e+51 erg, percent: 100.00%


## Problem 3

In [24]:

d_betelgues = 800 * u.lyr
d_betelgues = d_betelgues.to(u.cm)
print("Distance to Betelgeuse: {:.2e}".format(d_betelgues))

n_neutrinos = 1e57
neutrino_flux = n_neutrinos / (4 * np.pi * d_betelgues**2)
print("Neutrino flux: {:.2e}".format(neutrino_flux))

my_cross_section = 162 * 20 * u.cm**2
print("Cross section: {:.2e}".format(my_cross_section))
neutrinos_passing_through_me = neutrino_flux * my_cross_section
print("Neutrinos passing through me: {:.2e}".format(neutrinos_passing_through_me))

nucleon_cross_section = 1e-40 * u.cm**2

mass_human = 58 * u.kg
mass_proton = const.m_p.to(u.kg)
print(mass_human, mass_proton)
num_nucleons = (mass_human / mass_proton) 
print("Number of nucleons: {:.2e}".format(num_nucleons))
interaction_rate = nucleon_cross_section * neutrino_flux * num_nucleons
print("Interaction rate: {:.2e}".format(interaction_rate))


Distance to Betelgeuse: 7.57e+20 cm
Neutrino flux: 1.39e+14 1 / cm2
Cross section: 3.24e+03 cm2
Neutrinos passing through me: 4.50e+17
58.0 kg 1.67262192369e-27 kg
Number of nucleons: 3.47e+28
Interaction rate: 4.82e+02


In [25]:
solar_sigma = 1e-42 *u.cm**2
solar_neutrino_flux_at_1_au = 6.5e10 * u.cm**-2 *u.s**-1
interactions_per_second = solar_sigma * solar_neutrino_flux_at_1_au * num_nucleons
print("Interactions per second: {:.2e}".format(interactions_per_second))
print("Interactions per day: {:.2e}".format(interactions_per_second.to(u.day**-1)))
print("Interactions per year: {:}".format(interactions_per_second.to(u.year**-1)))


Interactions per second: 2.25e-03 1 / s
Interactions per day: 1.95e+02 1 / d
Interactions per year: 71129.13582857595 1 / yr


# Problem 5

In [26]:
extra_galactic_nova_distance = 10 * u.kpc
extra_galactic_flux = n_neutrinos / (4 * np.pi * extra_galactic_nova_distance**2)
print("extra galactic flux: {:.2e}".format(extra_galactic_flux.to(u.cm**-2) ))
ice_cub_detector = 1 * u.km**3 
water_density = 1 * u.g / u.cm**3
n_targets = water_density * ice_cub_detector.to(u.cm**3) / mass_proton.to(u.g)
print("Number of targets: {:.2e}".format(n_targets))    

num_ice_cube_interactions = n_targets * extra_galactic_flux.to(u.cm**-2) * nucleon_cross_section
print("Number of ICECUBE interactions: {:.2e}".format(num_ice_cube_interactions))

extra galactic flux: 8.36e+10 1 / cm2
Number of targets: 5.98e+38
Number of ICECUBE interactions: 5.00e+09


In [27]:
ice_cub_detector.to(u.cm**3) 

<Quantity 1.e+15 cm3>

In [28]:
5.14 * (0.12)**2 *(10/100)**2 * (10/10)**2

0.0007401600000000001

# Problem 6

In [82]:
def radius_main_sequence(mass):
    return 1.4 * mass.value**0.5 * u.Rsun

m_init = 10 * u.M_sun
r_init = radius_main_sequence(m_init)
print("r_init: {:.2e}".format(r_init))
period_init = 3 * u.day
omega_init = 2 * np.pi / period_init.to(u.s)

ang_mom_init = 0.4 * m_init * r_init**2 * omega_init
print("Initial radius: {:.2e}".format(r_init))
print("Initial period: {:.2e}".format(period_init.to(u.s)))
print("Initial angular momentum: {:.2e}".format(ang_mom_init.to(u.kg * u.m**2 / u.s)))

r_final = (10* u.km).to(u.Rsun)
m_final = 2 * u.M_sun

omega_final = ang_mom_init / (0.4 * m_final * r_final**2)


period_final = 2 * np.pi / omega_final
print("Final radius: {:.2e}".format(r_final))
print("Final omega: {:.2e}".format(omega_final.to(u.s**-1)))
print("Final period: {:.2e}".format(period_final.to(u.s)))
# pf = period_init *( r_final / r_init)**2
print("Final radius {:.2e}".format(r_final.to(u.m)))
speed_at_equator = 2* np.pi * r_final / period_final 
print("Speed at equator: {:.2e}c".format(speed_at_equator.to(u.m / u.s) / const.c))

gravity_at_equator = surface_grav(m_final, r_final)
print("Gravity at equator: {:.2e}".format(gravity_at_equator.to(u.m / u.s**2)))

# surface gravity with rotation
g_eff = gravity_at_equator  - r_final * omega_final**2
print("Effective gravity at equator: {:.2e}".format(g_eff.to(u.m / u.s**2)))

oblateness = (5/4) * (r_final.to(u.m)**3 * omega_final**2) / (const.G * m_final.to(u.kg))
print("Oblateness: {:.2e}".format(oblateness))


r_init: 4.43e+00 solRad
Initial radius: 4.43e+00 solRad
Initial period: 2.59e+05 s
Initial angular momentum: 1.83e+45 kg m2 / s
Final radius: 1.44e-05 solRad
Final omega: 1.15e+07 1 / s
Final period: 5.46e-07 s
Final radius 1.00e+04 m
Speed at equator: 3.84e+02c
Gravity at equator: 2.65e+12 m / s2
Effective gravity at equator: -1.32e+18 m / s2
Oblateness: 6.23e+05


In [38]:
u.M_sun.to(u.kg)

1.988409870698051e+30

# Question 8

In [51]:
def eddington_lum(mass):
    mass = mass.to(u.kg)
    thomson_scattering = const.sigma_T 
    return (4 * np.pi * const.G * mass * const.m_p * const.c) / thomson_scattering

mass_sag_a = 4.1e6 * u.Msun 
lum_sag_a = eddington_lum(mass_sag_a).to(u.erg/u.s)
print("Luminosity of Sag A*: {:.2e}".format(lum_sag_a))
print("Luminosity of Sag A*  {:.2e}".format(lum_sag_a.to(u.g * u.cm**2 / u.s**3)))

distance = 8200 * u.pc
flux = lum_sag_a / (4 * np.pi * distance.to(u.cm)**2)
print("Flux at Earth Sag A*: {:.2e}".format(flux.to(u.erg / u.s / u.cm**2)))

sirius_a_lum = 24.7 * u.Lsun
distance_sirius_a = 2.6 * u.pc
flux_sirius_a = sirius_a_lum.to(u.erg / u.s) / (4 * np.pi * distance_sirius_a.to(u.cm)**2)
print("Flux at Earth Sirius A: {:.2e}".format(flux_sirius_a.to(u.erg / u.s / u.cm**2)))

print("Flux ratio: {:.2e}".format(flux / flux_sirius_a))
efficiency = 0.1
mdot_edd = lum_sag_a / (const.c**2 * efficiency)
print("Eddington accretion rate: {:.2e}".format(mdot_edd.to(u.g / u.s)))
print("Eddington accretion rate: {:.2e}".format(mdot_edd.to(u.Msun / u.yr)))
    

Luminosity of Sag A*: 5.15e+44 erg / s
Luminosity of Sag A*  5.15e+44 cm2 g / s3
Flux at Earth Sag A*: 6.41e-02 erg / (cm2 s)
Flux at Earth Sirius A: 1.17e-04 erg / (cm2 s)
Flux ratio: 5.48e+02
Eddington accretion rate: 5.73e+24 g / s
Eddington accretion rate: 9.10e-02 solMass / yr


# Problem 9 

In [84]:
m_wd = 1    * u.Msun
m_rg =5 * u.Msun
period = 1.0 * u.yr
wd_mass_transfer = 1e-7 * u.Msun / u.yr 
omega = 2 * np.pi / period.to(u.s)
semi_major_axis = (const.G * ( m_rg + m_wd) * period**2 / (4 * np.pi**2))**(1/3) 
print("Semi-major axis: {:.2e}".format(semi_major_axis.to(u.au)))
a_wd = m_rg / (m_rg + m_wd) * semi_major_axis
print("Semi-major axis of the white dwarf: {:.2e}".format(a_wd.to(u.au)))
a_rg = m_wd / (m_rg + m_wd) * semi_major_axis
print("Semi-major axis of the red giant: {:.2e}".format(a_rg.to(u.au)))

v_orbital_rg =  2 * np.pi * a_rg / period.to(u.s)
v_orbital_wd = 2 * np.pi * a_wd / period.to(u.s)
print("Orbital velocity of the red giant: {:.2e}".format(v_orbital_rg.to(u.km / u.s)))
print("Orbital velocity of the white dwarf: {:.2e}".format(v_orbital_wd.to(u.km / u.s)))

Semi-major axis: 1.82e+00 AU
Semi-major axis of the white dwarf: 1.51e+00 AU
Semi-major axis of the red giant: 3.03e-01 AU
Orbital velocity of the red giant: 9.02e+00 km / s
Orbital velocity of the white dwarf: 4.51e+01 km / s


In [85]:
v_wind = 10 * u.km / u.s
v_rel_wd = np.sqrt(v_wind**2 + v_orbital_wd**2)
print("Relative velocity of the wind: {:.2e}".format(v_rel_wd.to(u.km / u.s)))
bondi_accretion_radius = (2 * const.G * m_wd) / (v_rel_wd**2)
print("Bondi accretion radius: {:.2e}".format(bondi_accretion_radius.to(u.cm)))
fraction_accreted= bondi_accretion_radius**2 / (4 * semi_major_axis**2)
print("Fraction of mass accreted: {:.2}".format(fraction_accreted.to(u.dimensionless_unscaled)))

Relative velocity of the wind: 4.62e+01 km / s
Bondi accretion radius: 1.24e+13 cm
Fraction of mass accreted: 0.052


## using Bondi

In [88]:
rho = wd_mass_transfer / (4 * np.pi * semi_major_axis**2 * v_wind)
mdot_bondi = 4 * np.pi * const.G**2 * m_wd**2 * rho / v_rel_wd**3 
print("Bondi accretion rate: {:.2e}".format(mdot_bondi.to(u.Msun / u.yr)))

fraction_accreted = mdot_bondi / wd_mass_transfer
print("Fraction of mass accreted: {:.2}".format(fraction_accreted.to(u.dimensionless_unscaled)))

Bondi accretion rate: 2.42e-08 solMass / yr
Fraction of mass accreted: 0.24


# Problem 9

In [75]:
bol_lum = 3e4 * u.Lsun
bol_mag = -2.5 * np.log10(bol_lum / (const.L_sun.to(u.erg / u.s))) + 4.83
print("Bolometric magnitude: {:.2f}".format(bol_mag))

d_andromeda = 765 * u.kpc
abs_mag = bol_mag - 5 * np.log10((d_andromeda.to(u.pc) / 10*u.pc).value)
print("Absolute magnitude: {:.2f}".format(abs_mag))

Bolometric magnitude: -6.36
Absolute magnitude: -30.78
